# Demonstrating the Migration Matrix Calculator

This notebook shows how to use the `calculate_migration_matrix` function from `irbstudio.simulation.migration`.

The migration matrix is a critical component for simulating the behavior of an existing loan portfolio over time. It quantifies the probability of a loan moving from one rating to another in a single period.

The process is as follows:
1. **Create Sample Data**: We'll construct a pandas DataFrame representing historical observations of several loans, tracking their ratings over multiple dates.
2. **Calculate the Matrix**: We'll pass this data to the `calculate_migration_matrix` function.
3. **Visualize the Matrix**: We'll use Plotly to create a heatmap of the resulting matrix, which provides an intuitive visual representation of the transition probabilities.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Add the project root to the Python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from irbstudio.simulation.migration import calculate_migration_matrix

pio.templates.default = "plotly_white"
print("Imports successful.")

Imports successful.


### 1. Create Sample Historical Data

This data represents a small portfolio of loans. Each row is an observation of a specific loan's rating on a given date.

In [2]:
data = {
    'loan_id': ['L1', 'L1', 'L1', 'L2', 'L2', 'L3', 'L3', 'L3', 'L4', 'L4', 'L5', 'L5'],
    'date': [
        '2022-01-31', '2022-02-28', '2022-03-31',  # L1: A -> B -> B
        '2022-01-31', '2022-02-28',              # L2: A -> A
        '2022-01-31', '2022-02-28', '2022-03-31',  # L3: B -> C -> B
        '2022-01-31', '2022-02-28',              # L4: C -> C
        '2022-01-31', '2022-02-28',              # L5: B -> D (Default)
    ],
    'rating': ['A', 'B', 'B', 'A', 'A', 'B', 'C', 'B', 'C', 'C', 'B', 'D']
}
portfolio_history = pd.DataFrame(data)

print("Sample Portfolio History:")
display(portfolio_history)

Sample Portfolio History:


,loan_id,date,rating
0,L1,2022-01-31,A
1,L1,2022-02-28,B
2,L1,2022-03-31,B
3,L2,2022-01-31,A
4,L2,2022-02-28,A
5,L3,2022-01-31,B
6,L3,2022-02-28,C
7,L3,2022-03-31,B
8,L4,2022-01-31,C
9,L4,2022-02-28,C


### 2. Calculate the Migration Matrix

We now pass the DataFrame to the function, specifying the columns for the loan identifier, date, and rating.

In [3]:
migration_matrix = calculate_migration_matrix(
    data=portfolio_history,
    id_col='loan_id',
    date_col='date',
    rating_col='rating'
)

print("Calculated Migration Matrix:")
display(migration_matrix.style.format("{:.2%}").background_gradient(cmap='Blues'))

Calculated Migration Matrix:


,A,B,C,D
A,50.00%,50.00%,0.00%,0.00%
B,0.00%,33.33%,33.33%,33.33%
C,0.00%,50.00%,50.00%,0.00%
D,0.00%,0.00%,0.00%,100.00%


#### Interpreting the Matrix:

The matrix shows the probability of moving from a rating (row) to another rating (column).

- **From 'A'**: 50% of the time, loans stayed in 'A', and 50% of the time they migrated to 'B'.
- **From 'B'**: 33% stayed in 'B', 33% migrated to 'C', and 33% migrated to 'D'.
- **From 'C'**: 50% stayed in 'C', and 50% migrated back to 'B'.
- **From 'D'**: 100% stayed in 'D', as it's a terminal state (e.g., default).

### 3. Visualize the Matrix as a Heatmap

In [4]:
fig = go.Figure(data=go.Heatmap(
    z=migration_matrix.values,
    x=migration_matrix.columns,
    y=migration_matrix.index,
    colorscale='Blues',
    text=migration_matrix.values,
    texttemplate="%{text:.2%}",
    textfont={"size":14}
))

fig.update_layout(
    title='Rating Migration Matrix',
    xaxis_title='To Rating',
    yaxis_title='From Rating',
    yaxis=dict(autorange='reversed') # Place 'A' at the top
)

fig.show()